# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [38]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [39]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
print(filepath)


# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):

# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
#     print(file_path_list)

/home/workspace
/home/workspace/event_data


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [40]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)   # skip the header of csv file
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            # print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows and list of event data rows
# print(len(full_data_rows_list))
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [41]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


### Show data in created file csv

In [42]:
import pandas as pd
df = pd.read_csv('event_datafile_new.csv')
df.head(3)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88
2,Building 429,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Majesty (LP Version),88


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [43]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
from cassandra.cluster import Cluster
cluster = Cluster()


# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [44]:
session.execute("""CREATE KEYSPACE IF NOT EXISTS thuchh_cassandra
                   WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}
                """)

#### Set Keyspace

In [45]:
session.set_keyspace('thuchh_cassandra')

## Create queries to answer the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [46]:
# CSV file will be used in 3 queries

file = 'event_datafile_new.csv'

In [47]:
# Show data in created file csv

import pandas as pd
df = pd.read_csv('event_datafile_new.csv')
df.head(3)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88
2,Building 429,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Majesty (LP Version),88


In [ ]:
## Query 1:  Give me the artist, song title and song's length
## sessionId = 338, and itemInSession = 4


# drop table need to be created if it existed
session.execute("DROP TABLE IF EXISTS song_features;")

# create table song_features for query 1
song_features = (
    """
    CREATE TABLE IF NOT EXISTS 
    song_features (itemInSession int, sessionId int, artist text, 
                    song text, length float, 
                    PRIMARY KEY(itemInSession, sessionId) 
                    );
    """
)

session.execute(song_features)


# insert data for song_features table
insert_data_song_features = """ INSERT INTO song_features (
                                     itemInSession, sessionId, 
                                     artist, song, length
                                     )
                                VALUES (%s, %s, %s, %s, %s);
                            """

# open the newly created csv file, and then write lines of the file to song_features table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip the header
    for line in csvreader:
        # assing csv columns in each variable
        itemInSession, sessionId  = int(line[3]), int(line[8])
        artist, title, length = str(line[0]), str(line[9]), float(line[5])
        
        # execute the insertion
        session.execute(insert_data_song_features, (itemInSession, sessionId, artist, title, length))


In [ ]:

################################ Testing ###################################
# test data uploaded to table song_features

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip the header
    for line in csvreader:
        # execute insert queries from queries.py and query columns from def get_data
        itemInSession, sessionId  = int(line[3]), int(line[8])
        artist, title, length = str(line[0]), str(line[9]), float(line[5])
        data_1 = itemInSession, sessionId, artist, title, length
        
        session.execute(insert_data_song_features, (data_1))

        
rows = session.execute("select * from song_features")
for row in rows[0:5]:
    print(row)

#### Do a SELECT to verify that the data have been inserted into each table

In [ ]:
# Check data if meet the qualification of query 1
## sessionId = 338, and itemInSession = 4

query_1 = """
             SELECT artist, song, length 
             FROM song_features 
             WHERE itemInSession = %s AND sessionId = %s;
          """

rows = session.execute(query_1, (4, 338))
for row in rows:
    print(f'artist: {row.artist}, song: {row.song} length: {row.length:.8}')  


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [ ]:
######################### Test ###########################
# Show data in created file csv

import pandas as pd
df = pd.read_csv('event_datafile_new.csv')
df.head(3)

In [ ]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)
## for userid = 10, sessionid = 182


# drop table need to be created if it existed
session.execute("DROP TABLE IF EXISTS artist_song_by_user;")

# create table artist_song_by_user for query 2
artist_song_by_user = """ CREATE TABLE IF NOT EXISTS artist_song_by_user (
                                 userId int, sessionId int, itemInSession int, 
                                 artist text, song text, firstName text, lastName text, 
                                 PRIMARY KEY ((userId, sessionId), itemInSession)
                                );
                      """

session.execute(artist_song_by_user)



# insert data for artist_song_by_user table
insert_data_artist_song_by_user = """ INSERT INTO artist_song_by_user (
                                             userId, sessionId, itemInSession, 
                                             artist, song, firstName, lastName) 
                                             VALUES (%s, %s, %s, %s, %s, %s, %s
                                             );
                                  """

# open the newly created csv file, and then write lines of the file to artist_song_by_user table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip the header
    for line in csvreader:
        # assing csv columns in each variable
        userId, sessionId, itemInSession  = int(line[10]), int(line[8]), int(line[3])
        artist, song, firstName, lastName = str(line[0]), str(line[9]), str(line[1]), str(line[4])
        
        # execute the insertion
        session.execute(insert_data_artist_song_by_user, (userId, sessionId, itemInSession, artist, song, firstName, lastName))
                    

In [ ]:
# Check data if meet the qualification of query 2
## for userid = 10, sessionid = 182

query_2 = """ 
              SELECT artist, song, firstName, lastName 
              FROM artist_song_by_user
              WHERE userId = %s AND sessionId = %s;
          """

rows = session.execute(query_2, (10, 182))
for row in rows:
    print(f'artist: {row.artist}, song: {row.song}, user first name: {row.firstname}, user last name: {row.lastname}')  

In [ ]:
## Query 3: Give me every user name (first and last) who listened to the song 'All Hands Against His Own'


# drop table need to be created if it existed
session.execute("DROP TABLE IF EXISTS user_name;")

# create table user_name for query 3
user_name = """ CREATE TABLE IF NOT EXISTS user_name (
                        song text, userId int, 
                        firstName text, lastName text, 
                        PRIMARY KEY (song, userId)
                        );
            """

session.execute(user_name)


# insert data for user_name table
insert_data_user_name = """ INSERT INTO user_name (
                                song, userId, 
                                firstName, lastName
                                ) 
                            VALUES (%s, %s, %s, %s);
                        """

# open the newly created csv file, and then write lines of the file to user_name table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # assing csv columns in each variable
        song, userId, = str(line[9]), int(line[10])
        firstName, lastName = str(line[1]), str(line[4])
        
        session.execute(insert_data_user_name, (song, userId, firstName, lastName))
                    

In [ ]:
# check data if meet the qualification of query 3
## song = 'All Hands Against His Own'


query_3 = """ 
              SELECT firstName, lastName 
              FROM user_name 
              WHERE song = %s;
          """

rows = session.execute(query_3, ('All Hands Against His Own',))
for row in rows:
    print(f'user first name: {row.firstname:>10},  user last name: {row.lastname}')       

### Drop the tables before closing out the sessions

In [ ]:
session.execute("DROP TABLE IF EXISTS song_features")
session.execute("DROP TABLE IF EXISTS artist_song_by_user")
session.execute("DROP TABLE IF EXISTS user_name")                                                

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()